In [1]:
import numpy as np
import tensorflow as tf


In [2]:
import os
from six.moves.urllib.request import urlopen

In [3]:
# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [4]:
if not os.path.exists(IRIS_TRAINING):
  raw = urlopen(IRIS_TRAINING_URL).read()
  with open(IRIS_TRAINING,'wb') as f:
    f.write(raw)

In [5]:
if not os.path.exists(IRIS_TEST):
  raw = urlopen(IRIS_TEST_URL).read()
  with open(IRIS_TEST,'wb') as f:
    f.write(raw)

In [6]:
# Load datasets.
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TRAINING,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=IRIS_TEST,
    target_dtype=np.int,
    features_dtype=np.float32)

In [7]:
training_set.data.shape

(120, 4)

In [8]:
training_set.target.shape

(120,)

In [9]:
test_set.data.shape

(30, 4)

In [10]:
test_set.target.shape

(30,)

In [11]:
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[4])]

In [12]:
# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[10, 20, 10],
                                        n_classes=3,
                                        model_dir="/tmp/iris_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_tf_random_seed': 1, '_model_dir': '/tmp/iris_model', '_save_summary_steps': 100, '_log_step_count_steps': 100, '_session_config': None, '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5}


In [13]:
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(training_set.data)},
    y=np.array(training_set.target),
    num_epochs=None,
    shuffle=True)

In [14]:
# Train model.
classifier.train(input_fn=train_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/iris_model\model.ckpt.
INFO:tensorflow:loss = 310.678, step = 1
INFO:tensorflow:global_step/sec: 409.015
INFO:tensorflow:loss = 14.7495, step = 101 (0.260 sec)
INFO:tensorflow:global_step/sec: 425.801
INFO:tensorflow:loss = 4.84119, step = 201 (0.219 sec)
INFO:tensorflow:global_step/sec: 476.024
INFO:tensorflow:loss = 12.5913, step = 301 (0.226 sec)
INFO:tensorflow:global_step/sec: 416.282
INFO:tensorflow:loss = 13.0455, step = 401 (0.232 sec)
INFO:tensorflow:global_step/sec: 432.595
INFO:tensorflow:loss = 3.72096, step = 501 (0.230 sec)
INFO:tensorflow:global_step/sec: 436.369
INFO:tensorflow:loss = 9.10482, step = 601 (0.229 sec)
INFO:tensorflow:global_step/sec: 409.544
INFO:tensorflow:loss = 9.85962, step = 701 (0.244 sec)
INFO:tensorflow:global_step/sec: 459.111
INFO:tensorflow:loss = 3.39733, step = 801 (0.215 sec)
INFO:tensorflow:global_step/sec: 471.689
INFO:tensorflow:loss = 4.65043, 

In [15]:
# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(test_set.data)},
    y=np.array(test_set.target),
    num_epochs=1,
    shuffle=False)

In [17]:
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))


INFO:tensorflow:Starting evaluation at 2017-12-11-18:30:14
INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-2000
INFO:tensorflow:Finished evaluation at 2017-12-11-18:30:15
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.966667, average_loss = 0.0535388, global_step = 2000, loss = 1.60617

Test Accuracy: 0.966667



In [18]:
# Classify two new flower samples.
new_samples = np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)


In [19]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": new_samples},
    num_epochs=1,
    shuffle=False)



In [20]:
predictions = list(classifier.predict(input_fn=predict_input_fn))


INFO:tensorflow:Restoring parameters from /tmp/iris_model\model.ckpt-2000


In [21]:
predicted_classes = [p["classes"] for p in predictions]



In [22]:
print(
    "New Samples, Class Predictions:    {}\n"
    .format(predicted_classes))

New Samples, Class Predictions:    [array([b'1'], dtype=object), array([b'2'], dtype=object)]

